# **Day18**: *Lavaduct Lagoon*

## Part 1
Thanks to your efforts, the machine parts factory is one of the first factories up and running since the lavafall came back. However, to catch up with the large backlog of parts requests, the factory will also need a large supply of lava for a while; the Elves have already started creating a large lagoon nearby for this purpose.

However, they aren't sure the lagoon will be big enough; they've asked you to take a look at the dig plan (your puzzle input). For example:

```
R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)
```

The digger starts in a 1 meter cube hole in the ground. They then dig the specified number of meters up (U), down (D), left (L), or right (R), clearing full 1 meter cubes as they go. The directions are given as seen from above, so if "up" were north, then "right" would be east, and so on. Each trench is also listed with the color that the edge of the trench should be painted as an RGB hexadecimal color code.

When viewed from above, the above example dig plan would result in the following loop of trench (#) having been dug out from otherwise ground-level terrain (.):

```
#######
#.....#
###...#
..#...#
..#...#
###.###
#...#..
##..###
.#....#
.######
```

At this point, the trench could contain 38 cubic meters of lava. However, this is just the edge of the lagoon; the next step is to dig out the interior so that it is one meter deep as well:

```
#######
#######
#######
..#####
..#####
#######
#####..
#######
.######
.######
```

Now, the lagoon can contain a much more respectable 62 cubic meters of lava. While the interior is dug out, the edges are also painted according to the color codes in the dig plan.

The Elves are concerned the lagoon won't be large enough; if they follow their dig plan, how many cubic meters of lava could it hold?

### Solution

If we start at point `(0,0)`, we can loop over the instructions to find the locations of the vertices that make up the lava lagoon. The area of an irregular polygon (polygon that is not a standard shape like triangle, square, etc.) can be calculated using the [shoelace method](https://en.wikipedia.org/wiki/Shoelace_formula).

$$
2A = 
\left|\begin{array}{cc} 
x_1 & x_2\\ 
y_1 & y_2
\end{array}\right|
+ 
\left|\begin{array}{cc} 
x_2 & x_3\\ 
y_2 & y_3
\end{array}\right|
+ 
...
+ 
\left|\begin{array}{cc} 
x_n & x_1\\ 
y_n & y_1
\end{array}\right|
$$ 

In [1]:
import numpy as np

def area_irregular_polygon(vertices):
    """
    Calculates the area of an irregular polygon defined by a set of vertices 
    using the shoelace method. 
    """
    x, y = map(list, list(zip(*vertices)))
    
    x.append(x[0])
    y.append(y[0])
    area = 0
    for i in range(len(vertices)):
        matrix = np.array([x[i:i+2], y[i:i+2]])
        area += np.linalg.det(matrix)
    
    return area/2

According to [Pick's theorem](https://en.wikipedia.org/wiki/Pick's_theorem), the area of a polygon with integer coordinates is given by 

$$ A = i + b/2 -1 $$

Here $i$ is the number of integer coordinates inside of the polygon, $b$ the number of integer coordinates on the polygon perimeter. From the instructions we can derrive the perimeter $b$, and using the above shoelace method we can find the area $A$. 

Because of the way the pit is being dug out, the value we are interested in is $i + b$, which can be obtained by rearranging the formula above

$$i+b = A + b/2 + 1$$

In [2]:
def calculate_area(input_string):
    lines = input_string.splitlines()
    instructions = [tuple(line.split(' ')[:2]) for line in lines]
    direction_map = {'U': (0, -1), 'D': (0, 1), 'R': (1, 0), 'L': (-1, 0)}
    
    x, y = 0, 0
    vertices = [(x, y)] 
    perimeter = 0
    for direction, distance in instructions:
        dx, dy = direction_map[direction] 
        x += dx*int(distance)
        y += dy*int(distance)
        perimeter += abs(dx*int(distance) + dy*int(distance))
        vertices.append((x, y))
    
    A = area_irregular_polygon(vertices)
    area = A + perimeter/2 + 1
    return int(round(area))
        


### Example

In [3]:
input_string = """R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)"""

calculate_area(input_string)


62

### Submission

In [4]:
with open('input/day18.txt', 'r') as file:
    input_string = file.read()

calculate_area(input_string)


56678

## Part 2

The Elves were right to be concerned; the planned lagoon would be much too small.

After a few minutes, someone realizes what happened; someone swapped the color and instruction parameters when producing the dig plan. They don't have time to fix the bug; one of them asks if you can extract the correct instructions from the hexadecimal codes.

Each hexadecimal code is six hexadecimal digits long. The first five hexadecimal digits encode the distance in meters as a five-digit hexadecimal number. The last hexadecimal digit encodes the direction to dig: 0 means R, 1 means D, 2 means L, and 3 means U.

So, in the above example, the hexadecimal codes can be converted into the true instructions:

```
#70c710 = R 461937
#0dc571 = D 56407
#5713f0 = R 356671
#d2c081 = D 863240
#59c680 = R 367720
#411b91 = D 266681
#8ceee2 = L 577262
#caa173 = U 829975
#1b58a2 = L 112010
#caa171 = D 829975
#7807d2 = L 491645
#a77fa3 = U 686074
#015232 = L 5411
#7a21e3 = U 500254
```

Digging out this loop and its interior produces a lagoon that can hold an impressive 952408144115 cubic meters of lava.

Convert the hexadecimal color codes into the correct instructions; if the Elves follow this new dig plan, how many cubic meters of lava could the lagoon hold?

### Solution

In [6]:
def calculate_area(input_string):   
    x, y = 0, 0
    vertices = [(x, y)] 
    perimeter = 0
      
    directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    for line in input_string.splitlines():
        code = line.split('(')[1][1:-1]
        distance = int(code[:-1], 16)
                
        dx, dy = directions[int(code[-1])]
        x += dx*int(distance)
        y += dy*int(distance)
        perimeter += abs(dx*int(distance) + dy*int(distance))
        
        vertices.append((x, y))
            
    A = area_irregular_polygon(vertices)
    area = A + perimeter/2 + 1
    return int(round(area))


### Example

In [7]:
input_string = """R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)"""

calculate_area(input_string)


952408144115

### Submission

In [8]:
with open('input/day18.txt', 'r') as file:
    input_string = file.read()

calculate_area(input_string)


79088855654037